<a href="https://colab.research.google.com/github/prashanthacha/Mechine-learning/blob/main/cyberbullying_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv("cyberbullying_tweets.csv.zip")
data.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [ ]:
data.isnull().sum()

,0
tweet_text,0
cyberbullying_type,0


In [ ]:
data.shape

(47692, 2)

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.isnull().sum()

,0
tweet_text,0
cyberbullying_type,0


In [ ]:
data.shape

(47692, 2)

In [ ]:
data.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [ ]:
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
data['cyberbullying_type']= label_encoder.fit_transform(data['cyberbullying_type'])

data['cyberbullying_type'].unique()

array([3, 2, 5, 4, 0, 1])

In [ ]:
X = data['tweet_text'].values
y = data['cyberbullying_type'].values

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
ps = PorterStemmer()
corpus = []
for i in range(len(X)):
    #print(i)
    news = re.sub('[^a-zA-Z]', ' ', X[i])
    news = news.lower()
    news = news.split()
    news = [ps.stem(word) for word in news if word not in stopwords.words('english')]
    news = ' '.join(news)
    corpus.append(news)

In [ ]:
X[0], corpus[0]

('In other words #katandandre, your food was crapilicious! #mkr',
 'word katandandr food crapilici mkr')

In [ ]:
!pip install imblearn
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline, make_pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.asarray(corpus), y, test_size = 0.2, random_state = 24)

In [ ]:
X_train.shape, X_test.shape

((38153,), (9539,))

In [ ]:
X_train[0], y_train[0]

(np.str_('obsurf teenag dodg rambl make assert assert zero evid refer quran hadith garbag'),
 np.int64(5))

In [ ]:
X_test[0], y_test[0]

(np.str_('iamth teacher far deep end good night'), np.int64(3))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

results=[]

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [ ]:
tfidf = TfidfVectorizer()
model = RandomForestClassifier()

In [ ]:
x_train = tfidf.fit_transform(X_train)
x_test = tfidf.transform(X_test)

In [ ]:
model = RandomForestClassifier(bootstrap=False, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None,
                       min_impurity_decrease=0.0,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
from imblearn.over_sampling import SMOTE
pipe_line = Pipeline([
                      ('tfidf', tfidf),
                      ('smote', SMOTE(random_state=12)),
                      ('model', model)
])

In [ ]:
pipe_line.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('smote', SMOTE(random_state=12)),
                ('model',
                 RandomForestClassifier(bootstrap=False,
                                        class_weight='balanced',
                                        n_estimators=250, random_state=42))])

In [ ]:
pred = pipe_line.predict(X_test)
print(accuracy_score(y_test, pred))
acc=accuracy_score(y_test, pred)
results.append(acc)

0.8274452248663382


In [ ]:
import pickle

In [ ]:
pickle.dump(pipe_line, open("randomforest.pkl", 'wb'))

In [ ]:
!pip install xgboost==1.1.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.6/127.6 MB 7.7 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.4
    Uninstalling xgboost-2.1.4:
      Successfully uninstalled xgboost-2.1.4


In [ ]:
import xgboost
from xgboost import XGBClassifier

/usr/local/lib/python3.11/dist-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 314, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))


In [ ]:
xgb_model = XGBClassifier()
xgb_pipe_line = Pipeline([
                      ('tfidf', tfidf),
                      ('smote', SMOTE(random_state=12)),
                      ('model', xgb_model)
])

In [ ]:
xgb_pipe_line.fit(X_train, y_train.astype(np.float32))


ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.

In [ ]:
pred = xgb_pipe_line.predict(X_test)
acc=accuracy_score(y_test, pred)
results.append(acc)

print(accuracy_score(y_test, pred))

NameError: name 'xgb_pipe_line' is not defined

In [ ]:
pickle.dump(xgb_pipe_line, open("xgb.pkl", 'wb'))

NameError: name 'xgb_pipe_line' is not defined

In [ ]:
xgboost.__version__

'1.1.1'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json

# creating the dataset
data = {'Random Forest':results[0],'Xgboost':results[1]}

courses = list(data.keys())
values = list(data.values())

fig = plt.figure(figsize = (10, 5))

colors = ['red', 'green']
# creating the bar plot
plt.bar(courses, values, color =colors,
        width = 0.4)

plt.xlabel("Random Forest Vs XGboost")
plt.ylabel("Accuracy")
plt.show()